In [1]:
import parallelproj
import torch
import numpy as np
from odl.contrib import fom
from torch import optim
import numpy as np
from array_api_compat import to_device
import array_api_compat.torch as xp
from data import *
from geometry import *
from torch.nn.parallel import DataParallel as DP

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/parallelproj/backend.py:18: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  from numpy.array_api._array_object import Array
/home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


In [2]:
torch.cuda.set_device(1)

In [3]:
proj = get_minipet_projector(dev,num_rings=21)
num_train = 500
batch_size = 6

train_loader = torch.utils.data.DataLoader(RandomEllipsoids(num_ellipsoids=np.random.poisson(10), diag=200, proj=proj, num_images=num_train, axes_scale=0.3, dim=3),
                                            batch_size, shuffle=True, num_workers=9)


/home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/parallelproj/pet_scanners.py:309: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  side = inds // self.num_lor_endpoints_per_side
/home/mamo_alegua/anaconda3/envs/anton2/lib/python3.9/site-packages/parallelproj/pet_lors.py:258: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

In [4]:
from lpdUnetCrossAttention3 import *

num_epochs = 150
n_iter = 3
normalisation=(proj.norm(xp,"cuda"))**2
model = DP(LPD(n_iter, proj, normalisation,indices=patch_sinogram(proj, patch_size=(7,7), return_values=True)), device_ids=[1,2,3,4,6,7])

model.to(dev)
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters())))

for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

no_decay = []
decay = []
for name, param in model.named_parameters():
    if 'weight' in name and 'batchnorm' and 'layernorm' and 'relative_bias_table' not in name:
        decay.append(param)
    else:
        no_decay.append(param)

optimizer = optim.AdamW([
    {'params': decay, 'weight_decay': 0.01},
    {'params': no_decay, 'weight_decay': 0}
], lr=1e-4)

loss_function = nn.MSELoss()
lr_scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-4,
                                             steps_per_epoch=len(train_loader), epochs=num_epochs)

start_epoch = 0

scanner = None
lor_desc = None
torch.cuda.empty_cache()

Number of parameters: 19,596,686


In [5]:
#cp = torch.load("/home/mamo_alegua/anaconda3/LPDTransformer/"+"3iterUnetCA125")
#model.module.load_state_dict(cp['model'])
#optimizer.load_state_dict(cp['optimizer'])
#lr_scheduler.load_state_dict(cp['lr_scheduler'])
#start_epoch = cp['epoch'] + 1
#del cp

In [6]:
torch.backends.cudnn.benchmark = True

from torch.cuda.amp import GradScaler
scaler = GradScaler(growth_interval=100)

for epoch in range(start_epoch, num_epochs):
    running_loss = 0.0
    ssim_list = []
    psnr_list = []
    for idx, images in enumerate(train_loader, 0):
        images = to_device(images, dev)
        with torch.autocast(device_type="cuda"):
            trainSino = generate_data(blur_image(images, kernel_size=5, sigma=2.0), proj, noise_level=np.random.uniform(0.1,1.2))
        
            for param in model.parameters():
                param.grad = None
                
            outputs = model(trainSino)

            loss_value = loss_function(outputs, images)
        
        images = images.to('cpu').numpy()
        del trainSino

        scaler.scale(loss_value).backward()
        scaler.step(optimizer)
        scale = scaler.get_scale()
        scaler.update()

        skip_lr_sched = (scale > scaler.get_scale())
        if not skip_lr_sched:
            lr_scheduler.step()

        running_loss += loss_value.item()

        for j in range(outputs.shape[0]):
            for i in range(proj.in_shape[1]):
                psnr_list.append(fom.psnr(outputs[j,:,i,:].detach().cpu().numpy(),images[j,:,i,:]))
                ssim_list.append(fom.ssim(outputs[j,:,i,:].detach().cpu(),images[j,:,i,:]))

    running_loss /= len(train_loader)

    psnr_value = np.mean(psnr_list)
    ssim_value = np.mean(ssim_list)

    print("{}/{} - loss {:.3f}; PSNR {:.3f}; SSIM {:.3f}".format(
        epoch+1, num_epochs, running_loss, psnr_value, ssim_value))
    
    if (epoch+1) % 25 == 0:
        print("Saving Checkpoint...")
        cp = {'epoch': epoch,'model': model.module.state_dict(), 'optimizer': optimizer.state_dict(), 'lr_scheduler': lr_scheduler.state_dict()}
        torch.save(cp, "/home/mamo_alegua/anaconda3/LPDTransformer/"+"3iterUnetV2CA"+str(epoch+1))

1/150 - loss 0.729; PSNR -inf; SSIM 0.000
2/150 - loss 0.362; PSNR 3.036; SSIM 0.002
3/150 - loss 0.191; PSNR 5.795; SSIM 0.007
4/150 - loss 0.092; PSNR 8.991; SSIM 0.016
5/150 - loss 0.038; PSNR -inf; SSIM 0.037
6/150 - loss 0.014; PSNR -inf; SSIM 0.083
7/150 - loss 0.008; PSNR -inf; SSIM 0.154
8/150 - loss 0.006; PSNR -inf; SSIM 0.214
9/150 - loss 0.005; PSNR -inf; SSIM 0.250
10/150 - loss 0.004; PSNR -inf; SSIM 0.275
11/150 - loss 0.004; PSNR 22.780; SSIM 0.305
12/150 - loss 0.004; PSNR 23.067; SSIM 0.327
13/150 - loss 0.004; PSNR -inf; SSIM 0.337
14/150 - loss 0.003; PSNR -inf; SSIM 0.381
15/150 - loss 0.003; PSNR -inf; SSIM 0.396
16/150 - loss 0.003; PSNR 24.142; SSIM 0.400
17/150 - loss 0.003; PSNR 24.478; SSIM 0.426
18/150 - loss 0.002; PSNR 24.860; SSIM 0.455
19/150 - loss 0.003; PSNR -inf; SSIM 0.466
20/150 - loss 0.002; PSNR 25.047; SSIM 0.483
21/150 - loss 0.002; PSNR 25.397; SSIM 0.496
22/150 - loss 0.002; PSNR -inf; SSIM 0.462
23/150 - loss 0.002; PSNR -inf; SSIM 0.517
24/